### Helper Functions For Preprocessing Data

In [1]:
import gzip
import json
import joblib
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler

def load_json_gz_to_dataframe(file_path):
    """
    Load gzipped JSON data into a DataFrame.
    """
    data = []
    with gzip.open(file_path, 'rt') as f:
        for line in f:
            json_data = json.loads(line)
            for transcript, positions in json_data.items():
                for position, sequences in positions.items():
                    position = int(position)
                    for sequence, reads in sequences.items():
                        data.append({
                            'transcript_id': transcript,
                            'position': position,
                            'sequence': sequence,
                            'reads': reads
                        })
    return pd.DataFrame(data)

def extract_mean_reads(dataset):
    """
    Compute the mean of 'reads' for each row.
    """
    dataset['mean_reads'] = dataset['reads'].apply(lambda x: np.mean(x, axis=0))
    return dataset

def scale_mean_reads(dataset, scaler=None, scaler_path='mean_reads_scaler.pkl'):
    """
    Scale the 'mean_reads' column using StandardScaler.
    """
    if scaler is None: # If no scaler is provided, fit a new one and save it
        scaler = StandardScaler()
        scaled_mean_reads = scaler.fit_transform(np.vstack(dataset['mean_reads'].values))
        dataset['scaled_mean_reads'] = list(scaled_mean_reads)
        joblib.dump(scaler, scaler_path)
        print('Scaler saved to', scaler_path)
    else: # Use the provided scaler to transform the data
        scaled_mean_reads = scaler.transform(np.vstack(dataset['mean_reads'].values))
        dataset['scaled_mean_reads'] = list(scaled_mean_reads)
    return dataset

def load_scaler(scaler_path='mean_reads_scaler.pkl'):
    """
    Load the saved scaler from the given path.
    """
    return joblib.load(scaler_path)

def drach_encoder():
    """
    Return a OneHotEncoder object with predefined DRACH motifs.
    """
    # Define DRACH motifs to be used for one-hot encoding
    D, R, A, C, H = ['A', 'G', 'T'], ['A', 'G'], ['A'], ['C'], ['A', 'C', 'T']
    drach_motifs = [d + r + a + c + h for d in D for r in R for a in A for c in C for h in H]
    encoder = OneHotEncoder(categories=[drach_motifs], handle_unknown='ignore')
    return encoder

def extract_middle_sequence(dataset):
    """
    Extract the middle 5-mers sequence from the 'sequence' column.
    """
    dataset['middle_sequence'] = dataset['sequence'].apply(lambda x: x[1:-1])
    return dataset

def one_hot_encode_DRACH(dataset, encoder=None, encoder_path='drach_encoder.pkl'):
    """
    Apply one-hot encoding to the middle 5-mers sequence
    """
    # One-hot encode the middle sequence
    if encoder is None: # If no encoder is provided, fit a new one and save it
        encoder = drach_encoder()
        one_hot_matrix = encoder.fit_transform(dataset[['middle_sequence']])
        joblib.dump(encoder, encoder_path)
        print('DRACH Encoder saved to', encoder_path)
    else:
        one_hot_matrix = encoder.transform(dataset[['middle_sequence']])
    dataset['middle_sequence_OHE'] = list(one_hot_matrix.toarray())
    return dataset

def load_DRACH_encoder(encoder_path='drach_encoder.pkl'):
    """
    Load the saved DRACH encoder from the given path.
    """
    return joblib.load(encoder_path)

def combine_data(dataset, labels):
    """
    Combine dataset with labels
    """
    # Left join dataset with labels on 'transcript_id' and 'position'
    merged_df = pd.merge(dataset, labels,
                         left_on=['transcript_id', 'position'],
                         right_on=['transcript_id', 'transcript_position'],
                         how='left')
    # Reorder gene_id to the first column and drop duplicate columns
    gene_id = merged_df['gene_id']
    merged_df = merged_df.drop(columns=['transcript_position', 'gene_id'])
    merged_df.insert(0, 'gene_id', gene_id)
    return merged_df

def prepare_for_model(dataset):
    """
    Combine 'scaled_mean_reads' and `middle_sequence_OHE` for model input.
    """
    combined_features = np.hstack([np.vstack(dataset['scaled_mean_reads']), np.vstack(dataset['middle_sequence_OHE'])])
    return combined_features

### Preprocessing

Load Data

In [2]:
labels = pd.read_csv('data.info.labelled')
labels.head(3)

,gene_id,transcript_id,transcript_position,label
0,ENSG00000004059,ENST00000000233,244,0
1,ENSG00000004059,ENST00000000233,261,0
2,ENSG00000004059,ENST00000000233,316,0


In [3]:
df = load_json_gz_to_dataframe('dataset0.json.gz')
df.head(3)

,transcript_id,position,sequence,reads
0,ENST00000000233,244,AAGACCA,"[[0.00299, 2.06, 125.0, 0.0177, 10.4, 122.0, 0..."
1,ENST00000000233,261,CAAACTG,"[[0.0126, 1.95, 111.0, 0.0125, 1.27, 108.0, 0...."
2,ENST00000000233,316,GAAACAG,"[[0.00432, 2.02, 104.0, 0.00299, 3.56, 99.3, 0..."


Assign labels to the data

In [ ]:
df = combine_data(df, labels)
df.head(3)

Extract mean reads

In [92]:
df = extract_mean_reads(df)
df.head(3)

,gene_id,transcript_id,position,sequence,reads,label,mean_reads
0,ENSG00000004059,ENST00000000233,244,AAGACCA,"[[0.00299, 2.06, 125.0, 0.0177, 10.4, 122.0, 0...",0,"[0.008264378378378385, 4.223783783783786, 123...."
1,ENSG00000004059,ENST00000000233,261,CAAACTG,"[[0.0126, 1.95, 111.0, 0.0125, 1.27, 108.0, 0....",0,"[0.006609244186046515, 3.2164244186046504, 109..."
2,ENSG00000004059,ENST00000000233,316,GAAACAG,"[[0.00432, 2.02, 104.0, 0.00299, 3.56, 99.3, 0...",0,"[0.0075699999999999995, 2.94054054054054, 105...."


Train test split by gene_id

In [93]:
# Train test split by gene id
from sklearn.model_selection import train_test_split
train_gene_ids, test_gene_ids = train_test_split(df['gene_id'].unique(), test_size=0.2, random_state=4262)

train_df = df[df['gene_id'].isin(train_gene_ids)].copy()
test_df = df[df['gene_id'].isin(test_gene_ids)].copy()

Scale mean reads

In [94]:
# Scale mean reads of train data first
train_df = scale_mean_reads(train_df)
train_df.head(3)

Scaler saved to mean_reads_scaler.pkl


,gene_id,transcript_id,position,sequence,reads,label,mean_reads,scaled_mean_reads
18,ENSG00000003056,ENST00000000412,355,GAAACTA,"[[0.00232, 2.41, 109.0, 0.0222, 2.85, 111.0, 0...",0,"[0.007340399999999998, 2.9771799999999997, 108...","[-0.43079389735504386, -0.7090163040809951, -0..."
19,ENSG00000003056,ENST00000000412,367,GGGACCG,"[[0.00232, 1.32, 117.0, 0.0073, 7.89, 120.0, 0...",0,"[0.00898787234042553, 3.961489361702128, 118.6...","[0.49106279705280276, -0.1994143266757514, 0.6..."
20,ENSG00000003056,ENST00000000412,496,AGGACTG,"[[0.00398, 2.46, 111.0, 0.016, 3.36, 125.0, 0....",0,"[0.011064705882352935, 7.299607843137254, 115....","[1.6531720821786586, 1.5288144651450055, 0.418..."


In [95]:
# Load the saved scaler to transform the test data
scaler = load_scaler()
test_df = scale_mean_reads(test_df, scaler=scaler)

Extract middle sequence

In [96]:
train_df = extract_middle_sequence(train_df)
train_df.head(3)

,gene_id,transcript_id,position,sequence,reads,label,mean_reads,scaled_mean_reads,middle_sequence
18,ENSG00000003056,ENST00000000412,355,GAAACTA,"[[0.00232, 2.41, 109.0, 0.0222, 2.85, 111.0, 0...",0,"[0.007340399999999998, 2.9771799999999997, 108...","[-0.43079389735504386, -0.7090163040809951, -0...",AAACT
19,ENSG00000003056,ENST00000000412,367,GGGACCG,"[[0.00232, 1.32, 117.0, 0.0073, 7.89, 120.0, 0...",0,"[0.00898787234042553, 3.961489361702128, 118.6...","[0.49106279705280276, -0.1994143266757514, 0.6...",GGACC
20,ENSG00000003056,ENST00000000412,496,AGGACTG,"[[0.00398, 2.46, 111.0, 0.016, 3.36, 125.0, 0....",0,"[0.011064705882352935, 7.299607843137254, 115....","[1.6531720821786586, 1.5288144651450055, 0.418...",GGACT


In [103]:
test_df = extract_middle_sequence(test_df)

One hot encode middle sequence (DRACH motif)

In [104]:
train_df = one_hot_encode_DRACH(train_df)
train_df.head(3)

DRACH Encoder saved to drach_encoder.pkl


,gene_id,transcript_id,position,sequence,reads,label,mean_reads,scaled_mean_reads,middle_sequence,middle_sequence_OHE
18,ENSG00000003056,ENST00000000412,355,GAAACTA,"[[0.00232, 2.41, 109.0, 0.0222, 2.85, 111.0, 0...",0,"[0.007340399999999998, 2.9771799999999997, 108...","[-0.43079389735504386, -0.7090163040809951, -0...",AAACT,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19,ENSG00000003056,ENST00000000412,367,GGGACCG,"[[0.00232, 1.32, 117.0, 0.0073, 7.89, 120.0, 0...",0,"[0.00898787234042553, 3.961489361702128, 118.6...","[0.49106279705280276, -0.1994143266757514, 0.6...",GGACC,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
20,ENSG00000003056,ENST00000000412,496,AGGACTG,"[[0.00398, 2.46, 111.0, 0.016, 3.36, 125.0, 0....",0,"[0.011064705882352935, 7.299607843137254, 115....","[1.6531720821786586, 1.5288144651450055, 0.418...",GGACT,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [112]:
# Load the saved DRACH encoder to transform the test data
encoder = load_DRACH_encoder()
test_df = one_hot_encode_DRACH(test_df, encoder=encoder)

Check label imbalance (FOR SMOTE)

In [26]:
# Proportion of positive labels in the training set
np.mean(train_df['label'])

0.04520309457830705

In [42]:
# Proportion of positive labels by gene_id
prop_gene_id = train_df.groupby('gene_id')['label'].mean()

# Number of zeros
num_zeros = len(prop_gene_id[prop_gene_id == 0])
print('Number of gene_id with no postive labels: ', num_zeros)
print('Proportion of gene_id with no positive labels: ', num_zeros / len(prop_gene_id))

Number of gene_id with no postive labels:  1885
Proportion of gene_id with no positive labels:  0.6118143459915611


In [45]:
# Proportion of positive labels by middle_sequence
prop_drach = train_df.groupby('middle_sequence')['label'].mean()

# Number of zeros
num_zeros = len(prop_drach[prop_drach == 0])
print('Number of DRACH motifs with no postive labels: ', num_zeros)
print('Proportion of DRACH motifs with no positive labels: ', num_zeros / len(prop_drach))

Number of DRACH motifs with no postive labels:  0
Proportion of DRACH motifs with no positive labels:  0.0


In [62]:
# Number of positive labels by middle_sequence
train_df[train_df['label'] == 1].groupby('middle_sequence')['label'].count().sort_values()


middle_sequence
TAACA       1
TAACC       2
AAACC      10
AAACA      32
AGACC      51
TAACT      53
TGACA      59
TGACC      79
GAACC      95
AGACA     111
GAACA     136
AAACT     209
TGACT     355
GGACC     417
AGACT     429
GGACA     550
GAACT     653
GGACT    1146
Name: label, dtype: int64

Apply SMOTE to each group of DRACH motif

In [107]:
from imblearn.over_sampling import SMOTE

# Get the scaled mean reads and one-hot encoded middle sequence columns as dataframe
X_resampled_list = [] 
y_resampled_list = []
middle_sequence_list = []

for middle_seq, group in train_df.groupby('middle_sequence'):
    X = np.vstack(group['scaled_mean_reads'].values)
    y = group['label'].values
    
    if sum(y) < 10: # Skip if the number of positive labels is less than 10
        X_resampled_list.append(X)
        y_resampled_list.append(y)
        middle_sequence_list.extend([middle_seq] * len(y))
        continue

    # Apply SMOTE to the scaled mean reads and labels
    smote = SMOTE(random_state=4262)
    X_resampled, y_resampled = smote.fit_resample(X, y)

    # Append resampled data and one-hot encoding to lists
    X_resampled_list.append(X_resampled)
    y_resampled_list.append(y_resampled)
    middle_sequence_list.extend([middle_seq] * len(y_resampled))

# Combine all resampled data into final arrays
X_resampled = np.vstack(X_resampled_list)
y_resampled = np.concatenate(y_resampled_list)

# Combine into a final DataFrame if needed
resampled_df = pd.DataFrame({
    'scaled_mean_reads': list(X_resampled),
    'middle_sequence': middle_sequence_list,
    'label': y_resampled
})

In [81]:
# Proportion of positive labels in the resampled data
print('Proportion of positive labels in the resampled data: ', np.mean(resampled_df['label']))

# Proportion of positive labels by middle_sequence
prop_drach_resampled = resampled_df.groupby('middle_sequence')['label'].mean()
prop_drach_resampled

Proportion of positive labels in the resampled data:  0.483202765619337


middle_sequence
AAACA    0.500000
AAACC    0.500000
AAACT    0.500000
AGACA    0.500000
AGACC    0.500000
AGACT    0.500000
GAACA    0.500000
GAACC    0.500000
GAACT    0.500000
GGACA    0.500000
GGACC    0.500000
GGACT    0.500000
TAACA    0.000271
TAACC    0.000856
TAACT    0.500000
TGACA    0.500000
TGACC    0.500000
TGACT    0.500000
Name: label, dtype: float64

One hot encode DRACH motifs

In [108]:
resampled_df = one_hot_encode_DRACH(resampled_df)
resampled_df.head(3)

DRACH Encoder saved to drach_encoder.pkl


,scaled_mean_reads,middle_sequence,label,middle_sequence_OHE
0,"[2.5249757619759317, -0.7835678231993425, -0.2...",AAACA,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"[0.6928199968356539, -1.1676038905620205, -0.7...",AAACA,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[0.3732826597732115, -0.8372266235169966, -0.2...",AAACA,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


Prepare for model input

In [114]:
# With SMOTE
X_resampled = prepare_for_model(resampled_df)
y_resampled = resampled_df['label'].values

# Without SMOTE
X_train = prepare_for_model(train_df)
y_train = train_df['label'].values

# Prepare test data
X_test = prepare_for_model(test_df)
y_test = test_df['label'].values

### NN model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.metrics import AUC
from imblearn.over_sampling import SMOTE

def build_model(input_shape):
    """
    Define and compile neural network model.
    """
    model = Sequential([
        Input(shape=(input_shape,)),
        Dense(150, activation='relu'),
        Dropout(0.2),  # Dropout layer for regularization
        Dense(32, activation='relu'),
        Dropout(0.2),  # Another dropout layer
        Dense(1, activation='sigmoid')
    ])
    # Set AUC with Precision-Recall (PR) curve
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[AUC(curve='PR', name='auc_pr')])
    return model

#### Train model with SMOTE resampled data

In [116]:
# Define the model
SMOTE_model = build_model(X_resampled.shape[1])

# Train the model
checkpoint = ModelCheckpoint(
    'best_model_with_SMOTE.keras',  
    save_best_only=True,
    monitor='val_auc_pr',
    mode='max'
)

SMOTE_model.fit(
    X_resampled, y_resampled,
    epochs=5,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[checkpoint]
)

Epoch 1/5
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - auc_pr: 0.7365 - loss: 0.5676 - val_auc_pr: 0.2857 - val_loss: 0.4646
Epoch 2/5
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - auc_pr: 0.8509 - loss: 0.4513 - val_auc_pr: 0.3265 - val_loss: 0.4173
Epoch 3/5
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - auc_pr: 0.8738 - loss: 0.4161 - val_auc_pr: 0.3356 - val_loss: 0.4224
Epoch 4/5
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - auc_pr: 0.8869 - loss: 0.3912 - val_auc_pr: 0.3257 - val_loss: 0.3260
Epoch 5/5
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - auc_pr: 0.8943 - loss: 0.3787 - val_auc_pr: 0.2949 - val_loss: 0.3887


#### Train model without SMOTE

In [117]:
# Define the model
model = build_model(X_resampled.shape[1])

# Train the model
checkpoint = ModelCheckpoint(
    'best_model_without_SMOTE.keras',
    save_best_only=True,
    monitor='val_auc_pr',
    mode='max'
)

model.fit(
    X_resampled, y_resampled,
    epochs=5,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[checkpoint]
)

Epoch 1/5
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - auc_pr: 0.7437 - loss: 0.5642 - val_auc_pr: 0.3281 - val_loss: 0.4085
Epoch 2/5
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - auc_pr: 0.8504 - loss: 0.4524 - val_auc_pr: 0.3191 - val_loss: 0.3841
Epoch 3/5
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - auc_pr: 0.8727 - loss: 0.4180 - val_auc_pr: 0.3400 - val_loss: 0.3910
Epoch 4/5
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - auc_pr: 0.8869 - loss: 0.3927 - val_auc_pr: 0.3179 - val_loss: 0.3821
Epoch 5/5
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - auc_pr: 0.8947 - loss: 0.3787 - val_auc_pr: 0.3415 - val_loss: 0.4006
